In [5]:
import napari
import os
import numpy as np
import pandas as pd
from napari.settings import get_settings
import dask
import dask.array
from readlif.reader import LifFile
from dask import delayed
import dask.array as da
import torch
from cellpose import models

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
lif_file = 'R:\Betsy\Thunder_Ovcar\\20240213_OC_TC_2rounds\\20240213_Timecourse OVCAR8.lif'

image_stack = LifFile(lif_file)

In [13]:
img_list = [image.name for image in image_stack.get_iter_image() if (('Merged' in image.name))]
len(img_list)

28

In [14]:
img_list

['R1_RB-AF750/B/5_Merged',
 'R1_RB-AF750/B/7_Merged',
 'R1_RB-AF750/D/3_Merged',
 'R1_RB-AF750/D/4_Merged',
 'R1_RB-AF750/D/6_Merged',
 'R1_RB-AF750/C2_Merged',
 'R1_RB-AF750/F8_Merged',
 'R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/B/5_Merged',
 'R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/B/7_Merged',
 'R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/D/3_Merged',
 'R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/D/4_Merged',
 'R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/D/6_Merged',
 'R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/C2_Merged',
 'R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/F8_Merged',
 'R2_DAPI_Fibrillarin-AF647/B/5_Merged',
 'R2_DAPI_Fibrillarin-AF647/B/7_Merged',
 'R2_DAPI_Fibrillarin-AF647/D/3_Merged',
 'R2_DAPI_Fibrillarin-AF647/D/4_Merged',
 'R2_DAPI_Fibrillarin-AF647/D/6_Merged',
 'R2_DAPI_Fibrillarin-AF647/C2_Merged',
 'R2_DAPI_Fibrillarin-AF647/F8_Merged',
 'TileScan 1/B/5_Merged',
 'TileScan 1/B/7_Merged',
 'TileScan 1/D/3_Merged',
 'TileScan 1/D/4_Merged',
 'TileScan 1/D/6_Merged',
 'TileSc

In [8]:
# list all DAPI merged images
img_list = [[image.name, image.dims] for image in image_stack.get_iter_image() if (('Merged' in image.name) and ('DAPI' in image.name))]
img_list

[['R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/B/5_Merged',
  Dims(x=16752, y=16781, z=1, t=1, m=1)],
 ['R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/B/7_Merged',
  Dims(x=16751, y=16769, z=1, t=1, m=1)],
 ['R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/D/3_Merged',
  Dims(x=16763, y=16783, z=1, t=1, m=1)],
 ['R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/D/4_Merged',
  Dims(x=16754, y=16771, z=1, t=1, m=1)],
 ['R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/D/6_Merged',
  Dims(x=16756, y=16769, z=1, t=1, m=1)],
 ['R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/C2_Merged',
  Dims(x=16746, y=16768, z=1, t=1, m=1)],
 ['R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/F8_Merged',
  Dims(x=16755, y=16772, z=1, t=1, m=1)],
 ['R2_DAPI_Fibrillarin-AF647/B/5_Merged',
  Dims(x=16743, y=16776, z=1, t=1, m=1)],
 ['R2_DAPI_Fibrillarin-AF647/B/7_Merged',
  Dims(x=16761, y=16773, z=1, t=1, m=1)],
 ['R2_DAPI_Fibrillarin-AF647/D/3_Merged',
  Dims(x=16754, y=16784, z=1, t=1, m=1)],
 ['R2_DAPI_Fibrillarin-AF647/D/4_Merged',
  Dims(x=16761,

In [15]:
# example of getting the first image

def frame_getter(lif):

    return np.array(lif.get_frame(z=0, t=0))

sel_im = img_list[0][0] #change first [0] to [i] and loop through image list
print(sel_im)
dims = img_list[0][1] #change first [0] to [i] and loop through image list
print(dims)

im_lif = [image for image in image_stack.get_iter_image() if sel_im in image.name][0]
im_del = delayed(frame_getter)(im_lif)
# get the image in dask
im = da.from_delayed(im_del,shape=[dims.y,dims.x], dtype='uint16')
print('Output:')
print(type(im))
print(im.shape)

R1_DAPI_PLK1-AF488_PH2AX-AF555_p21-AF647/B/5_Merged
Dims(x=16752, y=16781, z=1, t=1, m=1)
Output:
<class 'dask.array.core.Array'>
(16781, 16752)


In [ ]:
# test run 
# model = models.Cellpose(model_type='cyto2',gpu=True)
# masks, _, _, _ = model.eval(im[5000:6000,5000:6000], diameter=80)

# viewer = napari.Viewer()
# viewer.add_image(im[5000:6000,5000:6000],contrast_limits=[0,10000])
# viewer.add_labels(masks)

In [28]:
# segment with cellpose
# on GPU on CTRL runs < 3min

model = models.Cellpose(model_type='cyto2',gpu=True)
masks, _, _, _ = model.eval(im, diameter=80)

In [29]:
# preview image
viewer = napari.Viewer()
viewer.add_image(im,contrast_limits=[0,10000])
viewer.add_labels(masks)

<Labels layer 'masks' at 0x2120144b3a0>

In [ ]:
# segmentation in the loop

for ind in range(len(img_list)):

    sel_im = img_list[ind][0]
    print(sel_im)
    dims = img_list[ind][1]
    print(dims)

    im_lif = [image for image in image_stack.get_iter_image() if sel_im in image.name][0]
    im_del = delayed(frame_getter)(im_lif)
    # get the image in dask
    im = da.from_delayed(im_del,shape=[dims.y,dims.x], dtype='uint16')

    # segmentation
    masks, _, _, _ = model.eval(im, diameter=80)

    # save mask

